In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# System sortowania owoców   
Model YOLOv8 do wykrywania uszkodzonych i zepsutych owoców.


In [ ]:
!pip install ultralytics torch torchvision torchaudio opencv-python numpy matplotlib pandas pyyaml tqdm


In [ ]:
from ultralytics import YOLO
from pathlib import Path
import cv2, os, yaml, random
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
from pathlib import Path
import yaml

data_root = Path("/kaggle/input/owoceyolo/OwoceYOLO")

class_names = [
    'fresh_apple',      # 0
    'apple_stem',       # 1
    'rotten_apple',     # 2
    'fresh_banana',     # 3
    'rotten_banana',    # 4
    'fresh_orange',     # 5
    'rotten_orange'     # 6
]

fruit_yaml = {
    'path': str(data_root),
    'train': 'images/train',
    'val': 'images/val',
    'names': {i: n for i, n in enumerate(class_names)}
}

with open('fruit.yaml', 'w') as f:
    yaml.safe_dump(fruit_yaml, f, sort_keys=False)

print(open('fruit.yaml').read())


In [ ]:
from pathlib import Path

data_root = Path("/kaggle/input/owoceyolo/OwoceYOLO")

for split in ['train', 'val']:
    img_dir = data_root / 'images' / split
    lbl_dir = data_root / 'labels' / split

    num_imgs = len(list(img_dir.glob('*')))
    num_lbls = len(list(lbl_dir.glob('*')))

    print(f"{split}: images = {num_imgs}, labels = {num_lbls}")


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(
    data="fruit.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=0.01,
    patience=20,
    device=0
)


In [ ]:
val_metrics = model.val()
print(val_metrics)


In [ ]:
from IPython.display import Image, display
import glob

for path in glob.glob("runs/detect/train*/val_batch*.jpg"):
    display(Image(filename=path))


In [ ]:
from ultralytics import YOLO
import torch

REJECT_CLASSES = {
    'rotten_apple',
    'rotten_banana',
    'rotten_orange'
}

WARN_CLASSES = {
    'apple_stem'
}

CONF_THRESH = 0.5

def decide_acceptance(result):
    reject, warn = False, False
    for b in result.boxes:
        cls = int(b.cls.cpu().numpy())
        cls_name = result.names[cls]
        conf = float(b.conf.cpu().numpy())

        if conf >= CONF_THRESH:
            if cls_name in REJECT_CLASSES:
                reject = True
            elif cls_name in WARN_CLASSES:
                warn = True

    if reject:
        return "REJECT"
    elif warn:
        return "ACCEPT_WITH_WARNING"
    return "ACCEPT"
